In [12]:
from astropy.io import ascii, fits
import numpy as np
import os
import glob
from astropy.time import Time
import matplotlib.pyplot as plt
from photutils.aperture import CircularAperture,CircularAnnulus,aperture_photometry
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
from astropy.modeling.models import Gaussian2D
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import sigma_clip, gaussian_fwhm_to_sigma
import matplotlib as mpl


#####INPUT#########################    
path = os.path.join('BetaCas') 
FILE = glob.glob(os.path.join(path,'pSQ*.fits'))
FILE = sorted(FILE)

Ann = 300 #[pix]
Dan = 200 #[pix]
Apersize = 4 # Aperture size = Apersize*FWHM
for n,file in enumerate(FILE):
    hdul = fits.open(file)[0]
    header = hdul.header
    data =hdul.data
    print(file.split('/')[-1], header['OBJECT'],header['HWPANG'])
    if (n+1)%8==0:
        print()
#########################    

pSQ20240926_13_37_53_Cam1.fits BetaCas_R 0.0
pSQ20240926_13_37_53_Cam2.fits BetaCas_R 0.0
pSQ20240926_13_37_53_Cam3.fits BetaCas_R 0.0
pSQ20240926_13_37_53_Cam4.fits BetaCas_R 0.0
pSQ20240926_13_38_07_Cam1.fits BetaCas_R 45.0
pSQ20240926_13_38_07_Cam2.fits BetaCas_R 45.0
pSQ20240926_13_38_07_Cam3.fits BetaCas_R 45.0
pSQ20240926_13_38_07_Cam4.fits BetaCas_R 45.0



In [7]:
def skyvalue(data,y0,x0,r_in,r_out):
    # Determine sky and std
    y_in = int(y0-r_out)
    y_out = int(y0+r_out)
    x_in = int(x0-r_out)
    x_out = int(x0+r_out)

    sky_deriving_area = data[y_in:y_out, x_in:x_out]
    mask = np.zeros(np.shape(sky_deriving_area))+1
    for yi in range(len(sky_deriving_area)):
        for xi in range(len(sky_deriving_area[0])):
            position = (xi - r_out)**2 + (yi-r_out)**2
            if position < (r_out)**2 and position > r_in**2 and xi > int(len(mask[0])/2):
                mask[yi, xi] = 0
    mask = mask.astype(bool)
    Sky_region = np.ma.masked_array(sky_deriving_area, mask)
    std = np.ma.std(Sky_region)
    sky = np.ma.median(Sky_region)
    npix = np.shape(sky_deriving_area)[0]*np.shape(sky_deriving_area)[1] - np.sum(mask)
    return(sky, std, npix)
def crop(data,row,col,size):
    row_str = int(row-size)
    row_end = int(row+size)
    col_str = int(col-size)
    col_end = int(col+size)
    data_cr = data[row_str:row_end,col_str:col_end]
    return data_cr
def circle(x,y,r):
    theta = np.linspace(0, 2*np.pi, 100)
    x1 = r*np.cos(theta)+y
    x2 = r*np.sin(theta)+x
    return(x2.tolist(),x1.tolist())

In [ ]:
#Apterture Photometry

Photo_Log = pd.DataFrame({})
order = np.arange(0,len(FILE),8)
for n_,z in enumerate(order):
    SET = [FILE[z],FILE[z+1], FILE[z+2], FILE[z+3],
           FILE[z+4],FILE[z+5], FILE[z+6], FILE[z+7]]
    JD_av = []
    EXP_av = []
    fig,ax = plt.subplots(2,4,figsize=(30,13))
    for i in range(0,8):
        File_ang = SET[i]  #Bring the fits file
        print(File_ang)
        hdul = fits.open(File_ang)
        header = hdul[0].header 
        data = hdul[0].data
        gain = header['GAIN']
        Filter = header['FILTER']
        RN = 1.26 
        
        
        #Fitting 2D Gaussian to the target
        center = pd.read_csv(File_ang+'.csv')
        x_init,y_init = center['XCENTER'].values[0],center['YCENTER'].values[0]-10
        #Find the FWHM of the object

        crop_size = 300
        crop_image = crop(data,y_init,x_init,crop_size)
        sky,sky_std,area = skyvalue(crop_image,crop_size,crop_size,
                                    crop_size/2,crop_size)
        crop_image_sub = crop_image - sky
        
        ##Fitting Gaussian2D
        g_init = Gaussian2D(amplitude=crop_image_sub[crop_size,crop_size],
                                x_mean = crop_size,y_mean=crop_size,
                                theta = 0)
        y, x = np.mgrid[:len(crop_image), :len(crop_image[0])]
        fitter = LevMarLSQFitter()
        fitted = fitter(g_init, x, y, crop_image_sub)
        center_x = fitted.x_mean.value
        center_y = fitted.y_mean.value
            
        ##Re-Fitting Gaussian2D
        re_g_init = Gaussian2D(amplitude=fitted.amplitude.value,
                               x_mean = center_x,y_mean=center_y,
                               theta = 0,                                   
                               x_stddev = fitted.x_stddev.value,
                               y_stddev = fitted.y_stddev.value)
        fitter = LevMarLSQFitter()
        fitted = fitter(re_g_init, x, y,crop_image_sub)
        center_x = fitted.x_mean.value
        center_y = fitted.y_mean.value
        fwhm = max(fitted.x_fwhm,fitted.y_fwhm)
        x_cen = center_x + (x_init-crop_size) 
        y_cen = center_y + (y_init-crop_size) 
            
            
        #Deriving the sky
        sky,sky_std,sky_npix = skyvalue(data,y_cen,x_cen,
                                        Ann,Ann+Dan) 
        #Aperture photometry
        aperture_radius = Apersize*1/2*fwhm
        aperture_ = CircularAperture([x_cen,y_cen],aperture_radius)
        Flux= aperture_photometry(data-sky, aperture_)['aperture_sum'][0]
        Flux_e, sky_e, sky_std_e = gain*Flux, gain*sky, gain*sky_std #[ADU]--> [e]
        err_Flux = np.sqrt(Flux_e + np.pi*aperture_radius**2*(sky_e + (RN)**2))
        
        
        lim=50
        fontsize=18
        figsize=400
        im = ax[i//4,i%4].imshow(data - sky,
                                 vmin=-2*lim,vmax=2*lim,cmap='seismic')
        xi,yi = circle(x_cen,y_cen,aperture_radius)
        ax[i//4,i%4].plot(xi,yi,color='k',lw=2.5,zorder=11)
        xi,yi = circle(x_cen,y_cen,Ann)
        ax[i//4,i%4].plot(xi,yi ,color='c',lw=2.5)
        xi,yi = circle(x_cen,y_cen,Ann+Dan)
        ax[i//4,i%4].plot(xi,yi ,color='c',lw=2.5)
        ax[i//4,i%4].plot(x_cen,y_cen,marker='X',ls='',color='k',ms=12)
        ax[i//4,i%4].plot(x_cen,y_cen,marker='x',ls='',color='yellow',ms=12)
        ax[i//4,i%4].plot(x_init, y_init, marker='x',ls='',color='c', ms=10)
        ax[i//4,i%4].set_xlim(x_cen-figsize,x_cen+figsize)
        ax[i//4,i%4].set_ylim(y_cen-figsize,y_cen+figsize)
        ax[i//4,i%4].set_title(File_ang.split('/')[-1]+'({0:.1f})'.format(float(header['HWPANG'])),
                               fontsize=12)
        divider = make_axes_locatable(ax[i//4,i%4])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im,cax=cax) 
        JD = Time(header['DATE-OBS']+'T'+header['UT'], format='isot').jd    
        Photo_Log = pd.concat([Photo_Log,
                               pd.DataFrame({
                                   'filename':[File_ang.split('/')[-1]],
                                   'Object':[header['OBJECT']],
                                   'DATE':[header['DATE-OBS']],
                                   'JD':[JD],
                                   'UT':[header['UT']],
                                   'HWPANG':[header['HWPANG']],
                                   'FILTER':[Filter],
                                   'EXPTIME':[header['EXPTIME']],
                                   'Cam':[i%4+1], #ID of cam (e.g., Cam1, Cam2, Cam3, Cam4)==0:
                                   'FWHM [pix]': [fwhm],
                                   'Aperture size [pix]':[aperture_radius],
                                   'Flux [e]':[Flux_e],
                                   'eFlux [e]':[err_Flux],
                                   'Sky [e]':[sky_e],
                                   'std_Sky [e]': [sky_std_e],
                                   'Ann [pix]':[Ann],
                                   'Dan [pix]':[Dan],
                                   'Cen (y,x)':[(int(y_cen),int(x_cen))]})])
#         plt.show()
    Phot_name = ['filename', 'Object','DATE','UT','JD', 'EXPTIME','Cam',
                 'HWPANG','FILTER','FWHM [pix]','Aperture size [pix]',
                 'Flux [e]','eFlux [e]','Sky [e]','std_Sky [e]','Ann [pix]',
                 'Dan [pix]','Cen (y,x)']
    Photo_Log = Photo_Log.reindex(columns=Phot_name)
    Photo_Log = Photo_Log.round({'EXPTIME':1,'HWPANG':1,
                                 'FWHM [pix]':1,'Aperture size [pix]':1,
                                 'Flux [e]':1,'eFlux [e]':1,'Sky [e]':1,'std_Sky [e]':1})
    Filename = os.path.join(path,'result','Photo_{0}.{1}.csv'.format(header['OBJECT'],header['DATE-OBS']))
    if os.path.exists(os.path.join(path,'result'))==False:
        os.mkdir(os.path.join(path,'result'))    
    Filename = Filename.replace('-','_')
    Photo_Log.to_csv(Filename, index=False)
        

BetaCas/pSQ20240926_13_37_53_Cam1.fits
BetaCas/pSQ20240926_13_37_53_Cam2.fits
BetaCas/pSQ20240926_13_37_53_Cam3.fits
BetaCas/pSQ20240926_13_37_53_Cam4.fits
BetaCas/pSQ20240926_13_38_07_Cam1.fits
